In [100]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [101]:
prescriptions = pd.read_parquet("../20_intermediate_files/prescriptions_wa.parquet", engine = "fastparquet")

In [102]:
prescriptions.columns

Index(['BUYER_STATE', 'BUYER_COUNTY', 'CountyFIPS_x', 'StateFIPS_x', 'Year',
       'MME', 'FIP_unique', 'County', 'State', 'Population', 'county_test',
       'state_abbrev', 'CountyFIPS_y', 'StateFIPS_y', 'CountyName', '_merge'],
      dtype='object')

In [103]:
prescriptions["State"].value_counts()

NC    600
CO    372
WA    234
MD    144
Name: State, dtype: int64

In [104]:
prescriptions["Year"].value_counts()

2009.0    225
2010.0    225
2011.0    225
2012.0    225
2013.0    225
2014.0    225
Name: Year, dtype: int64

In [105]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
# prescriptions_reduced_copy["DRUG_CODE"] = prescriptions_reduced_copy[
#     "DRUG_CODE"
# ].astype("int64")
# prescriptions_reduced_copy["Month"] = prescriptions_reduced_copy["Month"].astype(
#     "int64"
# )
# prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy[
#     "Population"
# ].astype("int64")

In [106]:
prescriptions_reduced_copy.head()

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
3,CO,adams,8001,8,2009,110355.414760,80018,Adams County,CO,435700.0,Adams,CO,8001.0,8.0,Adams,both
4,CO,adams,8001,8,2010,132304.866282,80018,Adams County,CO,443691.0,Adams,CO,8001.0,8.0,Adams,both
5,CO,adams,8001,8,2011,157909.630983,80018,Adams County,CO,452201.0,Adams,CO,8001.0,8.0,Adams,both
6,CO,adams,8001,8,2012,161238.256294,80018,Adams County,CO,460558.0,Adams,CO,8001.0,8.0,Adams,both
7,CO,adams,8001,8,2013,134536.857641,80018,Adams County,CO,469978.0,Adams,CO,8001.0,8.0,Adams,both


In [107]:
# creating a dataset that has all the drug prescriptions in the state of Washington

washington_prescriptions = prescriptions_reduced_copy[
    prescriptions_reduced_copy["State"] == "WA"
]

washington_prescriptions.head()

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge
index,,,,,,,,,,,,,,,,
3568,WA,adams,53001,53,2009,7673.547900,5300153,Adams County,WA,18405.0,Adams,WA,53001.0,53.0,Adams,both
3569,WA,adams,53001,53,2010,8516.658075,5300153,Adams County,WA,18790.0,Adams,WA,53001.0,53.0,Adams,both
3570,WA,adams,53001,53,2011,9310.998375,5300153,Adams County,WA,18877.0,Adams,WA,53001.0,53.0,Adams,both
3571,WA,adams,53001,53,2012,9063.213195,5300153,Adams County,WA,18944.0,Adams,WA,53001.0,53.0,Adams,both
3572,WA,adams,53001,53,2013,8987.487300,5300153,Adams County,WA,19098.0,Adams,WA,53001.0,53.0,Adams,both


In [144]:
washington_prescriptions_copy = washington_prescriptions.copy()
washington_prescriptions_copy["shipment_per_100k"] = (
        washington_prescriptions_copy["MME"]
    / (washington_prescriptions_copy["Population"])
    * 100_000
)

washington_prescriptions_copy.head()

,BUYER_STATE,BUYER_COUNTY,CountyFIPS_x,StateFIPS_x,Year,MME,FIP_unique,County,State,Population,county_test,state_abbrev,CountyFIPS_y,StateFIPS_y,CountyName,_merge,shipment_per_100k
index,,,,,,,,,,,,,,,,,
3568,WA,adams,53001,53,2009,7673.547900,5300153,Adams County,WA,18405.0,Adams,WA,53001.0,53.0,Adams,both,41692.735126
3569,WA,adams,53001,53,2010,8516.658075,5300153,Adams County,WA,18790.0,Adams,WA,53001.0,53.0,Adams,both,45325.482038
3570,WA,adams,53001,53,2011,9310.998375,5300153,Adams County,WA,18877.0,Adams,WA,53001.0,53.0,Adams,both,49324.566271
3571,WA,adams,53001,53,2012,9063.213195,5300153,Adams County,WA,18944.0,Adams,WA,53001.0,53.0,Adams,both,47842.130463
3572,WA,adams,53001,53,2013,8987.487300,5300153,Adams County,WA,19098.0,Adams,WA,53001.0,53.0,Adams,both,47059.835061


In [145]:
# calculating and displaying total number of drug prescriptions that took place in washington. results are grouped and displayed year and county wise 

wa_result = washington_prescriptions_copy.groupby(["Year"])["shipment_per_100k"].mean().reset_index()
# washington_prescriptions_result = washington_prescriptions.groupby(["Year"])["shipment_per_100k"].sum().reset_index()

wa_result.head(10)

,Year,shipment_per_100k
0,2009,75058.203038
1,2010,77468.976409
2,2011,77897.686034
3,2012,77317.159398
4,2013,77655.943149
5,2014,77489.839007


In [110]:
# change Year to be integer 
wa_result["Year"] = wa_result["Year"].astype("int")
wa_result.dtypes

Year                   int32
shipment_per_100k    float64
dtype: object

In [111]:
# split into before and after policy implementation 
wa_b4 = wa_result[wa_result["Year"] < 2012]
wa_after = wa_result[wa_result["Year"] >= 2012]

In [112]:
x = "Year"
y = "shipment_per_100k"

In [146]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):
   
    line = alt.Chart(pd.DataFrame({
    'Year': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Year:Q',
    color=alt.Color('color:N', scale=None)
    )
 
    return line


In [147]:
# creating the vertical line at 2012, the year of implementation of policy changes in Washington
line = vertical_line(2012)



In [152]:
# creating the regression model and calculating the error bands for creating the plots
 
def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):
   
    # Creating the grid for predicted values
    colour = color
    years = [2009,2010,2011,2012,2013,2014]
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})
 
    # Fitting the model and making the predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
   
    # Utilizing the predictions to create the points and error bands in the chart
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color = alt.Color("Before/After",legend=alt.Legend(title = "Before/After Policy Change"))
        )
    )
 
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Shipment (MME) Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart

In [153]:
# Using the get_reg_fit_and_ci function to create the charts
 
def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data, color=color, xvar=xvar, yvar=yvar,  legend=legend, alpha=alpha,
    )
    return reg_chart
 


In [154]:
# creating the final plot for pre-post analysis of opioid prescriptions in Washington 
 
washington_before_chart = build_chart(
    wa_b4, "orange", "Year","shipment_per_100k", "Washington Before",  alpha=0.05
)
 
washington_after_chart = build_chart(
    wa_after, "blue", "Year", "shipment_per_100k", "Washington After", alpha=0.05
)
 
washington_final_pre_post_prescriptions = (washington_before_chart + washington_after_chart + line).properties(title="Pre Post Analysis Of Opioid Overdose Deaths In Washington")
 
washington_final_pre_post_prescriptions


alt.LayerChart(...)

In [117]:
########################################################################################################################
# ends here for final report     

In [118]:
# def vertical_line(year):
#     """Function to plot a vertical line at year of policy implementation"""
#     line = alt.Chart(pd.DataFrame({
#     'Date': [year],
#     'color': ["black"]
#     })).mark_rule().encode(
#     x='Date:Q', # use q for "quantitative" - as per altair docs
#     color=alt.Color('color:N', scale=None)
#     )

#     return line

In [119]:
# # test function
# line = vertical_line(2004)

In [120]:
# def get_charts(b4, after, title_b4, title_after):
#     """
#     Function to plot the pre and post charts.
#     Will not use in final plot - used as a baseline for our fit charts later.

#     """

#     base_before = (
#         alt.Chart(b4)
#         .mark_point()
#         .encode(
#             y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
#             x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
#         )
#         .properties(title=title_b4)
        
#     )

#     base_after = (
#         alt.Chart(after)
#         .mark_point()
#         .encode(
#             y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
#             x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
#         )
#         .properties(title=title_after)
#     )

#     return base_before, base_after

In [121]:
# test the function
# may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# however, if we can't add a title to our fit/regression line charts, we may need to add them here
# base_before, base_after = get_charts(b4 = wa_b4, after = wa_after, title_b4 = "shipments before policy", title_after="shipments after policy")
# base_before + base_after

In [122]:
# # starting here in final report
# # no longer calculating base chart above - just adding in regression line at same time
# def get_preds(df, x, y):

#     # init new empty df for our predictions
#     predictions = pd.DataFrame()

#     # fit our model and predict values
#     model = smf.ols(f"{y} ~ {x}", data=df).fit()
#     model_predict = model.get_prediction(df[x])

#     # save predictions back to df, calculate confidence intervals
#     predictions["shipment_per_100k"] = model.predict(df[x])
#     predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=0.05)

#     # save original year columns to new predictions df
#     predictions["Year"] = df["Year"]
#     predictions["year relative to policy"] = df["year relative to policy"]
#     return predictions

In [123]:
# def get_charts(b4, after, title_b4, title_after, color):
#     """
#     Function to plot the pre and post charts.
#     Will not use in final plot - used as a baseline for our fit charts later.

#     """

#     base_before = (
#         alt.Chart(b4)
#         .mark_point()
#         .encode(
#             y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
#             x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
#         )
#         .properties(title=title_b4).transform_regression("year relative to policy", "shipment_per_100k")
#     .mark_line()
#     .encode(color=alt.value(color))
        
#     )

#     base_after = (
#         alt.Chart(after)
#         .mark_point()
#         .encode(
#             y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
#             x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
            
#         )
#         .properties(title=title_after).transform_regression("year relative to policy", "shipment_per_100k")
#     .mark_line()
#     .encode(color=alt.value(color))
#     )

#     return base_before, base_after

In [124]:
# # test the function
# # may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# # however, if we can't add a title to our fit/regression line charts, we may need to add them here
# base_before, base_after = get_charts(b4 = wa_b4_preds, after = wa_after_preds, title_b4 = "Opioid Shipments Before and After Policy Implementation in Washington", title_after="Shipments After Policy Implementation", color="red")
# base_before + base_after

In [125]:
#####################################
# updated code for final report ends here 

In [126]:
#prescriptions["QUANTITY"].describe()

In [127]:
#prescriptions_reduced = prescriptions[['QUANTITY', 'Year', 'StateFIPS', 'StateName', 'CountyFIPS', 'FIP_unique', 'Population','county_test']]

In [128]:
#prescriptions_reduced["StateName"].value_counts()

In [129]:
# washington = prescriptions_reduced[prescriptions_reduced["StateName"] == "Washington"]
# comp = prescriptions_reduced[prescriptions_reduced["StateName"] != "Washington"]

In [130]:
# wa_prescriptions = washington.copy()
# comp_prescriptions = comp.copy()

In [131]:
# wa_prescriptions["quantity_sum"] = wa_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')
# comp_prescriptions["quantity_sum"] = comp_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')

In [132]:
# wa_prescriptions = wa_prescriptions.drop('QUANTITY', axis=1)
# comp_prescriptions = comp_prescriptions.drop('QUANTITY', axis=1)

In [133]:
# wa_result = wa_prescriptions.drop_duplicates()
# comp_result = comp_prescriptions.drop_duplicates()

In [134]:
# wa_result["shipment_per_100k"] = wa_result["quantity_sum"] / wa_result["Population"] * 100_000
# comp_result["shipment_per_100k"] = comp_result["quantity_sum"] / comp_result["Population"] * 100_000

In [135]:
# wa_result.groupby("Year")["shipment_per_100k"].agg([np.mean, np.std])

In [136]:
# wa_result_b4 = wa_result[wa_result["Year"] < 2012]
# wa_result_after = wa_result[wa_result["Year"] >= 2012]

In [137]:

# # washington

# source_data = wa_result_b4

# plot_wa_b4 = alt.Chart(source_data).mark_point().encode(
#     y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
#     x=alt.X("Year:O", scale=alt.Scale(zero=False))
# ).transform_aggregate(
#     mean_shipment='mean(shipment_per_100k)',
#     groupby=["Year"]
# )

# plot_wa_b4

In [138]:
# fit_wa_b4 = plot_wa_b4.transform_regression('Year', 'mean_shipment',method="linear"
# ).mark_line()

# fit_wa_b4

In [139]:
# source_data = wa_result_after

# plot_wa_after = alt.Chart(source_data).mark_point().encode(
#     y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
#     x=alt.X("Year:O", scale=alt.Scale(zero=False))
# ).transform_aggregate(
#     mean_shipment='mean(shipment_per_100k)',
#     groupby=["Year"]
# )

# plot_wa_after

In [140]:
# fit_wa_after = plot_wa_after.transform_regression('Year', 'mean_overdose',method="linear"
# ).mark_line()

# fit_wa_after

In [141]:
# plot_wa_b4 + fit_wa_b4 + plot_wa_after + fit_wa_after